<a href="https://colab.research.google.com/github/JcorredorG/Activity-6-POO/blob/main/8_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# ================================
# Hotel - Interfaz gráfica en Colab con Gradio
# Celda única
# ================================

!pip install gradio -q

from dataclasses import dataclass, field
from datetime import datetime, date
from typing import Optional, List
import gradio as gr

# ---------- MODELO: Habitacion y Hotel ----------

@dataclass
class Habitacion:
    numero: int
    tarifa_diaria: float
    ocupada: bool = False
    nombre_huesped: Optional[str] = None
    documento_huesped: Optional[str] = None
    fecha_ingreso: Optional[date] = None

    def reservar(self, nombre: str, documento: str, fecha_ingreso: date):
        if self.ocupada:
            raise ValueError("La habitación ya está ocupada.")
        self.nombre_huesped = nombre
        self.documento_huesped = documento
        self.fecha_ingreso = fecha_ingreso
        self.ocupada = True

    def liberar(self, fecha_salida: date) -> float:
        if not self.ocupada or self.fecha_ingreso is None:
            raise ValueError("La habitación está disponible.")
        dias = (fecha_salida - self.fecha_ingreso).days
        if dias <= 0:
            dias = 1  # mínimo un día
        total = dias * self.tarifa_diaria
        # limpiar estado
        self.ocupada = False
        self.nombre_huesped = None
        self.documento_huesped = None
        self.fecha_ingreso = None
        return total


@dataclass
class Hotel:
    habitaciones: List[Habitacion] = field(default_factory=list)

    def __post_init__(self):
        if not self.habitaciones:
            for i in range(1, 11):
                tarifa = 120000.0 if i <= 5 else 160000.0
                self.habitaciones.append(Habitacion(i, tarifa))

    def buscar(self, numero: int) -> Habitacion:
        if numero < 1 or numero > len(self.habitaciones):
            raise ValueError("Número de habitación inválido.")
        return self.habitaciones[numero - 1]

    # --- métodos que se usaban en la interfaz y antes faltaban ---
    def reservar(self, numero: int, nombre: str,
                 documento: str, fecha_ingreso: date):
        hab = self.buscar(numero)
        hab.reservar(nombre, documento, fecha_ingreso)

    def registrar_salida(self, numero: int, fecha_salida: date) -> float:
        hab = self.buscar(numero)
        return hab.liberar(fecha_salida)
    # -------------------------------------------------------------

    def resumen_habitaciones(self) -> str:
        lineas = []
        for h in self.habitaciones:
            estado = "Ocupada" if h.ocupada else "Disponible"
            lineas.append(
                f"Habitación {h.numero} - {estado} - Tarifa: $ {h.tarifa_diaria:,.0f}"
            )
        return "\n".join(lineas)


# hotel global que mantiene el estado entre clics
hotel = Hotel()


# ---------- FUNCIONES PARA LA INTERFAZ ----------

def reservar_interfaz(num_hab, nombre, documento, fecha_str):
    try:
        if not documento or not fecha_str:
            return "ERROR: Documento y fecha de ingreso son obligatorios.", hotel.resumen_habitaciones()

        fecha_ingreso = datetime.strptime(fecha_str.strip(), "%d/%m/%Y").date()
        hotel.reservar(int(num_hab), nombre.strip(), documento.strip(), fecha_ingreso)
        msg = f"Reserva registrada correctamente en la habitación {num_hab}."
    except ValueError as e:
        msg = f"ERROR: {e}"
    except Exception as e:
        msg = f"ERROR inesperado: {e}"

    return msg, hotel.resumen_habitaciones()


def salida_interfaz(num_hab, fecha_str):
    try:
        if not fecha_str:
            return "ERROR: Debes ingresar la fecha de salida.", hotel.resumen_habitaciones()

        fecha_salida = datetime.strptime(fecha_str.strip(), "%d/%m/%Y").date()
        total = hotel.registrar_salida(int(num_hab), fecha_salida)
        msg = f"Salida registrada. Total a pagar habitación {num_hab}: $ {total:,.0f}"
    except ValueError as e:
        msg = f"ERROR: {e}"
    except Exception as e:
        msg = f"ERROR inesperado: {e}"

    return msg, hotel.resumen_habitaciones()


# ---------- INTERFAZ GRADIO ----------

opciones_habitaciones = [str(h.numero) for h in hotel.habitaciones]

with gr.Blocks() as demo:
    gr.Markdown("# Hotel - Gestión de habitaciones\nEjercicio 8.5 (Interfaz gráfica en Python)")

    with gr.Tab("Reservar"):
        gr.Markdown("### Reservar habitación")
        with gr.Row():
            num_hab_r = gr.Dropdown(opciones_habitaciones, label="Habitación")
            nombre_r = gr.Textbox(label="Nombre del huésped (opcional)")
        with gr.Row():
            doc_r = gr.Textbox(label="Documento del huésped")
            fecha_r = gr.Textbox(label="Fecha de ingreso (dd/mm/aaaa)")
        btn_r = gr.Button("Reservar")
        salida_r = gr.Textbox(label="Resultado", interactive=False)
        resumen_r = gr.Textbox(label="Estado actual de habitaciones", interactive=False,
                               value=hotel.resumen_habitaciones(), lines=8)

        btn_r.click(
            reservar_interfaz,
            inputs=[num_hab_r, nombre_r, doc_r, fecha_r],
            outputs=[salida_r, resumen_r]
        )

    with gr.Tab("Salida"):
        gr.Markdown("### Registrar salida de huésped")
        with gr.Row():
            num_hab_s = gr.Dropdown(opciones_habitaciones, label="Habitación")
            fecha_s = gr.Textbox(label="Fecha de salida (dd/mm/aaaa)")
        btn_s = gr.Button("Registrar salida")
        salida_s = gr.Textbox(label="Resultado", interactive=False)
        resumen_s = gr.Textbox(label="Estado actual de habitaciones", interactive=False,
                               value=hotel.resumen_habitaciones(), lines=8)

        btn_s.click(
            salida_interfaz,
            inputs=[num_hab_s, fecha_s],
            outputs=[salida_s, resumen_s]
        )

# Esto hace que aparezca la interfaz debajo de la celda
# y también te da un link para abrirla en otra pestaña
demo.launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://0ffb8ad13c13dedcf0.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
